In [28]:
# Structure Visualiser
from pathlib import Path
from ase.visualize import view
from ase.io import read
from IPython.display import display

structure_dir = "Carbon_Structures/Crystalline/Relaxed/Final Trajectory Frame"

def view_relaxed_structures(dir, struct_num):

    dir = Path(dir)

    files = []
    for file in dir.rglob('*'):
        
        if file.suffix != ".cif":
            print(f"Unrecognized file suffix: {file.suffix}")
            continue
        files.append(file)
    chosen_file = files[struct_num]
    atoms = read(chosen_file)
    print(chosen_file.name)
    display(view(atoms, viewer='x3d'))

view_relaxed_structures(structure_dir,19)

medium-omat-0_relaxed_C60_GAP_20.cif
